# Data Importing

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import os
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from skimage import io, transform
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, EarlyStopping,CSVLogger,ProgbarLogger
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D, BatchNormalization
from keras.regularizers import l2


In [88]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

**Changable_Parameter**

In [ ]:
#here image size is 256x256
IMAGE_SIZE = 200
#every time process 32 images (61x31)~1951
batch_size = 32
#how many color have in these pictures
CHANNELS = 3
#80x25~1951
EPOCHS = 50
NUM_CLASSES = 38
input_shape = (batch_size, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
patience=5
data_dir ="E:\\Research\\Augmented Data"
split_folder = "E:\\Reseach2\\Augmented Split"
model_name="EfficientNetV2B0"  

**Directory Creation and split**

In [ ]:
import os
import shutil
import random
import math

train_val_dir = os.path.join(split_folder, "train_validation_data")
test_dir = os.path.join(split_folder, "test_data")

def split_data(source_folder, split_folder, train_val_ratio=0.9):
    # Checking if the split folders already exist
    if os.path.exists(train_val_dir) and os.path.exists(test_dir):
        print("Split folders already exist. Skipping data split.")
        return
    
    # Creating split folders if they do not exist
    if not os.path.exists(split_folder):
        os.makedirs(split_folder)
    if not os.path.exists(train_val_dir):
        os.makedirs(train_val_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
    
    # Counting variables
    train_val_count = 0
    test_count = 0
    
    # Iterate through main folder
    for class_folder in os.listdir(source_folder):
        class_path = os.path.join(source_folder, class_folder)
        
        if os.path.isdir(class_path):
            train_val_class_path = os.path.join(train_val_dir, class_folder)
            test_class_path = os.path.join(test_dir, class_folder)
            
            if not os.path.exists(train_val_class_path):
                os.makedirs(train_val_class_path)
            if not os.path.exists(test_class_path):
                os.makedirs(test_class_path)
            #listing images
            images = [img for img in os.listdir(class_path) if img.endswith('.jpg') or img.endswith('.png')]
            random.shuffle(images)
            
            split_index = int(len(images) * train_val_ratio)
            train_val_images = images[:split_index]
            test_images = images[split_index:]
            
            for img in train_val_images:
                src = os.path.join(class_path, img)
                dst = os.path.join(train_val_class_path, img)
                shutil.copy(src, dst)
                train_val_count += 1
                
            for img in test_images:
                src = os.path.join(class_path, img)
                dst = os.path.join(test_class_path, img)
                shutil.copy(src, dst)
                test_count += 1
    
    print("Data split completed successfully.")
    print(f"Number of images in train_validation_data: {train_val_count}")
    print(f"Number of images in test_data: {test_count}")
    print(f"Total number of images: {train_val_count + test_count}")

split_data(data_dir, split_folder)


Split folders already exist. Skipping data split.


In [ ]:
import os
import shutil
import random
import tensorflow as tf

# Parameters
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.10+.03,  # 10% of the data will be used for validation
)

train_generator = datagen.flow_from_directory(
    train_val_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=batch_size,
    class_mode='categorical',  
    subset='training' 
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


validation_generator = datagen.flow_from_directory(
    train_val_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=batch_size,
    class_mode='categorical', 
    subset='validation' 
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=batch_size,
    class_mode='categorical'  
)


Found 66120 images belonging to 38 classes.
Found 9880 images belonging to 38 classes.
Found 66255 images belonging to 38 classes.


In [ ]:
images, labels = next(train_generator)

# Plot four random images from the batch
fig, axes = plt.subplots(1, 4, figsize=(15, 5))
for i in range(4):
    # Generate a random index
    random_index = np.random.randint(0, batch_size)
    
    # Plot the image
    axes[i].imshow(images[random_index])
    
    # Get the corresponding label
    label = labels[random_index]
    
    # Set the title as the class name
    class_name = train_generator.class_indices
    axes[i].set_title(list(class_name.keys())[np.argmax(label)])

plt.show()

# Building model

In [96]:
from tensorflow.keras.applications import EfficientNetV2B0,ResNet50,InceptionV3,NASNetMobile,EfficientNetV2L

model= EfficientNetV2B0(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
                 include_preprocessing = False)
                 
# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 Activation False
4 Conv2D False
5 BatchNormalization False
6 Activation False
7 Conv2D False
8 BatchNormalization False
9 Activation False
10 Conv2D False
11 BatchNormalization False
12 Conv2D False
13 BatchNormalization False
14 Activation False
15 Conv2D False
16 BatchNormalization False
17 Add False
18 Conv2D False
19 BatchNormalization False
20 Activation False
21 Conv2D False
22 BatchNormalization False
23 Conv2D False
24 BatchNormalization False
25 Activation False
26 Conv2D False
27 BatchNormalization False
28 Add False
29 Conv2D False
30 BatchNormalization False
31 Activation False
32 DepthwiseConv2D False
33 BatchNormalization False
34 Activation False
35 GlobalAveragePooling2D False
36 Reshape False
37 Conv2D False
38 Conv2D False
39 Multiply False
40 Conv2D False
41 BatchNormalization False
42 Conv2D False
43 BatchNormalization False
44 Activation False
45 DepthwiseConv2D False
46 BatchNormalization False
47 Acti

**Fine Tuning**

In [98]:
for layer in model.layers[:182]:
   layer.trainable = False
for layer in model.layers[182:]:
   layer.trainable = True

# print('Last block of the conv_base is now trainable')


for i, layer in enumerate(model.layers):
   print(i, layer.name, layer.trainable)

0 input_2 False
1 stem_conv False
2 stem_bn False
3 stem_activation False
4 block1a_project_conv False
5 block1a_project_bn False
6 block1a_project_activation False
7 block2a_expand_conv False
8 block2a_expand_bn False
9 block2a_expand_activation False
10 block2a_project_conv False
11 block2a_project_bn False
12 block2b_expand_conv False
13 block2b_expand_bn False
14 block2b_expand_activation False
15 block2b_project_conv False
16 block2b_project_bn False
17 block2b_add False
18 block3a_expand_conv False
19 block3a_expand_bn False
20 block3a_expand_activation False
21 block3a_project_conv False
22 block3a_project_bn False
23 block3b_expand_conv False
24 block3b_expand_bn False
25 block3b_expand_activation False
26 block3b_project_conv False
27 block3b_project_bn False
28 block3b_add False
29 block4a_expand_conv False
30 block4a_expand_bn False
31 block4a_expand_activation False
32 block4a_dwconv2 False
33 block4a_bn False
34 block4a_activation False
35 block4a_se_squeeze False
36 block

In [99]:

def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed on top of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation='relu',kernel_regularizer=l2(0.01))(top_model)  # Adding L2 regularization
    top_model = Dropout(0.5)(top_model) 
    top_model = Dense(num_classes, activation='softmax')(top_model)
    return top_model


In [100]:
from keras.models import Model
FC_Head = lw(model, NUM_CLASSES)
model = Model(inputs = model.input, outputs = FC_Head)
#print(model.summary())

# Training

In [102]:
from tensorflow.keras.callbacks import LearningRateScheduler


def scheduler(epoch, lr):
     if epoch < 5:
         return .0009
     else:
        return .0009

lr_scheduler = LearningRateScheduler(scheduler)

In [103]:
if not os.path.exists("Results"):
    os.makedirs("Results")
if not os.path.exists("Model"):
    os.makedirs("Model") 
csv_logger_path = os.path.join("Results", f"{model_name}_results.csv")
model_checkpoint_path = os.path.join("Model", f"{model_name}_Model.hdf5")

**CheckPoints**

In [105]:

csv_logger = CSVLogger(csv_logger_path)
early_stopping = EarlyStopping(monitor='val_loss', patience=patience)
check = ModelCheckpoint(model_checkpoint_path, verbose=1, save_best_only=True)
progbar_logger = ProgbarLogger()

In [106]:


model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
history = model.fit(train_generator,
                    epochs=EPOCHS, 
                    batch_size=batch_size,
                    validation_data=validation_generator,
                    verbose = 1,
                    callbacks = [early_stopping,lr_scheduler,check,csv_logger],
                    initial_epoch=0)

Epoch 1/50
2067/2067 [==============================] - ETA: 0s - loss: 1.6416 - accuracy: 0.6908
Epoch 1: val_loss improved from inf to 0.85839, saving model to Model\EfficientNetV2B0_Model.hdf5
2067/2067 [==============================] - 218s 103ms/step - loss: 1.6416 - accuracy: 0.6908 - val_loss: 0.8584 - val_accuracy: 0.8385 - lr: 9.0000e-04
Epoch 2/50
2067/2067 [==============================] - ETA: 0s - loss: 0.8710 - accuracy: 0.8360
Epoch 2: val_loss improved from 0.85839 to 0.77939, saving model to Model\EfficientNetV2B0_Model.hdf5
2067/2067 [==============================] - 211s 102ms/step - loss: 0.8710 - accuracy: 0.8360 - val_loss: 0.7794 - val_accuracy: 0.8580 - lr: 9.0000e-04
Epoch 3/50
2067/2067 [==============================] - ETA: 0s - loss: 0.6736 - accuracy: 0.8865
Epoch 3: val_loss improved from 0.77939 to 0.60806, saving model to Model\EfficientNetV2B0_Model.hdf5
2067/2067 [==============================] - 212s 103ms/step - loss: 0.6736 - accuracy: 0.8865 -

**saving models**

In [108]:
import pickle

pickle_path_json = os.path.join("Results", f"{model_name}_model_architecture.json")
pickle_model_path = os.path.join("Model", f"{model_name}_model_pickle.pkl")

# Save model architecture to JSON (only needed once)
model_architecture = model.to_json()
with open(pickle_path_json, 'w') as f:
    f.write(model_architecture)

# Save model weights using pickle
with open(pickle_model_path, 'wb') as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://88e3c504-9146-4d68-83b0-7c6a80eea8b5/assets


In [109]:
model.save(f'{model_name}_.h5')

In [110]:
# model = tf.keras.models.load_model(f'{model_name}_.h5')

In [111]:
# history_path = os.path.join("Results", f"{model_name}__training_history.xlsx")
# import pandas as pd
# history_df = pd.DataFrame(history_path.history)
# history_df.to_excel(history_path, index=False)

# Result

In [113]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

In [ ]:

plt.figure(figsize=(7, 4))
plt.plot((epochs ), history.history['accuracy'], label='Training Accuracy')
plt.plot((epochs ), history.history['val_accuracy'], label='Validation Accuracy')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(epochs, history.history['loss'], label='Training Loss')
plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()
        

In [117]:
score = model.evaluate(test_generator ,verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1]*100)

2071/2071 [==============================] - 1141s 551ms/step - loss: 0.1678 - accuracy: 0.9900
Test Loss: 0.16778916120529175
Test accuracy: 98.99629950523376


In [118]:
from sklearn.metrics import roc_curve, auc, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np


In [119]:
predicted_vals = model.predict(test_generator, steps = len(test_generator))
predicted_vals.shape  # number of test samples x number of classes to predict

(66255, 38)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Predicted probabilities
predicted_vals = model.predict(test_generator, steps=len(test_generator))
# Get predicted class indices
predicted_classes = np.argmax(predicted_vals, axis=1)

# Extract true labels
true_classes = test_generator.classes

# Map class indices to their names
class_labels = list(test_generator.class_indices.keys())

# Generate classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("Classification Report:")
print(report)


In [121]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(true_classes, predicted_classes)
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.0262


In [ ]:
#3D plot
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Replace these with your actual history values
epochs = range(len(history.history['accuracy']))
accuracy = history.history['accuracy']

# Create a meshgrid for 3D plotting
X, Y = np.meshgrid(epochs, epochs)  # Both axes represent epochs
Z = np.tile(accuracy, (len(epochs), 1))  # Repeat accuracy array to fit the grid

# Create the 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Add color bar for reference
fig.colorbar(surf, ax=ax)

# Add titles and labels
ax.set_title('3D Surface of Accuracy over Epochs')
ax.set_xlabel('Epochs')
ax.set_ylabel('Epochs')
ax.set_zlabel('Accuracy')

plt.show()


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure `predicted_vals` and `true_classes` are already available from earlier computations

# Convert true_classes to match the probabilities' shape
true_classes_broadcasted = np.array([true_classes for _ in range(predicted_vals.shape[1])]).T

# Create a DataFrame for seaborn
df = pd.DataFrame({
    'True Classes': true_classes_broadcasted.flatten(),
    'Predicted Probabilities': predicted_vals.flatten()
})

# Generate the violin plot
plt.figure(figsize=(12, 8))
sns.violinplot(x=df['True Classes'], y=df['Predicted Probabilities'])
plt.title('Violin Plot of True Classes vs Predicted Probabilities')
plt.xlabel('True Classes')
plt.ylabel('Predicted Probabilities')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report

# Assuming predictions and true_classes as previously defined
# Get class-wise precision, recall, f1-score
report = classification_report(true_classes, predicted_classes, output_dict=True)
df_report = pd.DataFrame(report).transpose()

# Remove last three rows (accuracy, macro avg, weighted avg)
df_report = df_report[:-3]  

# Set the style and color palette
sns.set(style="whitegrid")

# Create a colorful boxplot using a palette
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_report[['precision', 'recall', 'f1-score']],
            palette="Set3",  # Color palette for vibrancy
            linewidth=2.5)   # Thicker borders

# Customize the plot for a more colorful and clear display
plt.title('Colorful Box Plot of Precision, Recall, F1-Score by Class', fontsize=14)
plt.ylabel('Scores', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Optional: Add grid lines for readability
plt.grid(True, linestyle='--', alpha=0.6)

# Show the plot
plt.show()


In [ ]:
#area plot
import matplotlib.pyplot as plt

# Area plot for accuracy and loss over epochs
epochs = np.arange(len(history.history['loss']))
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.fill_between(epochs, accuracy, color="skyblue", alpha=0.4, label='Training Accuracy')
plt.fill_between(epochs, val_accuracy, color="orange", alpha=0.4, label='Validation Accuracy')

plt.title('Area Plot of Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [126]:
from sklearn.metrics import jaccard_score
import numpy as np

# Use `predicted_vals` as predictions from the model
# `predicted_vals` contains probabilities; take the argmax to get class predictions
predicted_classes = np.argmax(predicted_vals, axis=1)

# True class labels from the test generator
true_classes = test_generator.classes

# Compute Jaccard similarity for each class using the 'average' parameter
jaccard_macro = jaccard_score(true_classes, predicted_classes, average='macro')
jaccard_micro = jaccard_score(true_classes, predicted_classes, average='micro')

# Display results
print(f"Jaccard Similarity (Macro Average): {jaccard_macro}")
print(f"Jaccard Similarity (Micro Average): {jaccard_micro}")

Jaccard Similarity (Macro Average): 0.013266590370317901
Jaccard Similarity (Micro Average): 0.013259313635529455


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import jaccard_score

# Assuming you have the true classes and predicted classes
predicted_classes = np.argmax(predicted_vals, axis=1)
true_classes = test_generator.classes

# Get the number of classes
num_classes = len(test_generator.class_indices)

# Compute Jaccard similarity for each class
jaccard_per_class = []
for i in range(num_classes):
    true_binary = (true_classes == i).astype(int)
    predicted_binary = (predicted_classes == i).astype(int)
    jaccard = jaccard_score(true_binary, predicted_binary)
    jaccard_per_class.append(jaccard)

# Plot Jaccard similarity for each class
plt.figure(figsize=(10,6))
sns.barplot(x=list(test_generator.class_indices.keys()), y=jaccard_per_class, palette="viridis")
plt.title('Jaccard Similarity for Each Class')
plt.xlabel('Classes')
plt.ylabel('Jaccard Similarity')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [128]:
pip install scipy 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import matplotlib.pyplot as plt

# Extract metrics from training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

# Create a figure with two subplots for accuracy and loss
plt.figure(figsize=(14, 6))

# Plot training and validation accuracy
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
plt.plot(epochs, acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

# Plot training and validation loss
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Show the merged plots
plt.tight_layout()
plt.show()

In [130]:
import numpy as np
from scipy import stats

# Example accuracies or predictions from two models
model_1_accuracies = [0.85, 0.87, 0.88, 0.86, 0.90]  # Sample accuracies for Model 1
model_2_accuracies = [0.82, 0.84, 0.83, 0.85, 0.86]  # Sample accuracies for Model 2

# Perform an independent two-sample t-test (two-tailed by default)
t_statistic, p_value = stats.ttest_ind(model_1_accuracies, model_2_accuracies)

# Output the t-statistic and p-value
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpretation of results
alpha = 0.05  # significance level
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between the models.")
else:
    print("Fail to reject the null hypothesis: No significant difference between the models.")


t-statistic: 2.8736848324283986
p-value: 0.020711387488081177
Reject the null hypothesis: There is a significant difference between the models.


In [ ]:
#bubble plot
import matplotlib.pyplot as plt
import numpy as np

# Assuming history has loss and accuracy for models
epochs = np.arange(len(history.history['loss']))
loss = history.history['loss']
accuracy = history.history['accuracy']

# Bubble plot based on loss and accuracy
sizes = 1000 * (1 - np.array(loss))  # Scale bubble size by loss (higher loss = smaller bubbles)
colors = np.array(accuracy)  # Color based on accuracy

plt.scatter(epochs, accuracy, s=sizes, c=colors, alpha=0.5, cmap='plasma')
plt.title('Bubble Plot of Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.colorbar()
plt.show()